<a href="https://colab.research.google.com/github/palit-ishan/Traffic-and-Road-Signs-Object-Detection-Project/blob/main/Road_Signs_Detection_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train a custom object detection model for Road Sign Detection with TensorFlow Lite Model Maker

We will use the [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) to train a custom object detection model to detect Road Signs and put the TFLite model on a Raspberry Pi.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data required and will shorten the training time.


## Preparation

### Install the required packages
We will install the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library we will use for evaluation.

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.3/577.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install numpy==1.23.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import the required packages.

In [ ]:
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
import cv2

In [ ]:
import numpy as np

### Prepare the dataset - Mounting the Dataset in Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Training the object detection model

### Loading the dataset

* Images in `train_data` is used to train the custom object detection model. Here we use 745 images to train the model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before. Validation is done on 132 images.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'drive/MyDrive/Project_Test_Data/train_jpg',
    'drive/MyDrive/Project_Test_Data/train_jpg',
    ['trafficlight','stop','speedlimit','crosswalk']
)
val_data = object_detector.DataLoader.from_pascal_voc(
    'drive/MyDrive/Project_Test_Data/validate_jpg',
    'drive/MyDrive/Project_Test_Data/validate_jpg',
    ['trafficlight','stop','speedlimit','crosswalk']
)

In [ ]:
val_data.size

132

### Selecting a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite2 to train our model, as it is at a good tradeoff between Latency and Average Precision.

In [ ]:
spec = model_spec.get('efficientdet_lite2')
spec.config.autoaugment_policy = 'v0'

###Training the TensorFlow model with the training data.

* Setting `epochs = 50`, which means it will go through the training dataset 50 times. We will look at the validation accuracy during training and stop when we see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 16` here so you will see that it takes 46 steps to go through the 745 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec,batch_size = 16,train_whole_model=True, epochs=50, validation_data=val_data)

Epoch 1/50
46/46 [==============================] - 106s 1s/step - det_loss: 1.3580 - cls_loss: 0.9094 - box_loss: 0.0090 - reg_l2_loss: 0.0762 - loss: 1.4342 - learning_rate: 0.0140 - gradient_norm: 1.7366 - val_det_loss: 0.5136 - val_cls_loss: 0.3527 - val_box_loss: 0.0032 - val_reg_l2_loss: 0.0763 - val_loss: 0.5899
Epoch 2/50
46/46 [==============================] - 47s 1s/step - det_loss: 0.6388 - cls_loss: 0.4357 - box_loss: 0.0041 - reg_l2_loss: 0.0764 - loss: 0.7152 - learning_rate: 0.0200 - gradient_norm: 2.7643 - val_det_loss: 0.4537 - val_cls_loss: 0.2409 - val_box_loss: 0.0043 - val_reg_l2_loss: 0.0765 - val_loss: 0.5302
Epoch 3/50
46/46 [==============================] - 46s 999ms/step - det_loss: 0.5203 - cls_loss: 0.3436 - box_loss: 0.0035 - reg_l2_loss: 0.0767 - loss: 0.5969 - learning_rate: 0.0199 - gradient_norm: 3.4135 - val_det_loss: 0.3374 - val_cls_loss: 0.2682 - val_box_loss: 0.0014 - val_reg_l2_loss: 0.0769 - val_loss: 0.4143
Epoch 4/50
46/46 [==================

### Evaluating the model with the validation data.

After training the object detection model using the images in the training dataset, use the 132 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 2 step to go through the 132 images in the validation dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
val_data.size

132

In [ ]:
model.evaluate(val_data)

3/3 [==============================] - 24s 3s/step



{'AP': 0.64514923,
 'AP50': 0.78636855,
 'AP75': 0.7598101,
 'APs': 0.48135948,
 'APm': 0.8445239,
 'APl': 0.9590602,
 'ARmax1': 0.58477634,
 'ARmax10': 0.7166675,
 'ARmax100': 0.7193265,
 'ARs': 0.57237744,
 'ARm': 0.8945395,
 'ARl': 0.96666664,
 'AP_/trafficlight': 0.45070502,
 'AP_/stop': 0.66664034,
 'AP_/speedlimit': 0.86242515,
 'AP_/crosswalk': 0.6008265}

### Export as a TensorFlow Lite model.

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is [full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). This allows the TensorFlow Lite model to be smaller, run faster on Raspberry Pi CPU and also compatible with the Google Coral EdgeTPU.

In [ ]:
model.export(export_dir='.', tflite_filename='road_signs_efficientdet_lite2_aug_v0_b16_ep50.tflite')

### Evaluate the TensorFlow Lite model.

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite('road_signs_efficientdet_lite2_aug_v0_b16_ep50.tflite', val_data)

132/132 [==============================] - 1332s 10s/step



{'AP': 0.62906927,
 'AP50': 0.78477293,
 'AP75': 0.7391676,
 'APs': 0.46777886,
 'APm': 0.81886834,
 'APl': 0.9587459,
 'ARmax1': 0.56349117,
 'ARmax10': 0.67622435,
 'ARmax100': 0.67622435,
 'ARs': 0.52254903,
 'ARm': 0.8516886,
 'ARl': 0.96,
 'AP_/trafficlight': 0.44190758,
 'AP_/stop': 0.6226553,
 'AP_/speedlimit': 0.8511243,
 'AP_/crosswalk': 0.60059}

##Testing Model on a still image to see output with Bounding Box

In [ ]:
labels = ['trafficlight','stop','speedlimit','crosswalk']

In [ ]:
# Code to use model
interpreter = tf.lite.Interpreter(model_path="road_signs_efficientdet_lite2_aug_v0_b16_ep50.tflite")
interpreter.allocate_tensors()
signature_fn = interpreter.get_signature_runner()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [ ]:
input_details

[{'name': 'serving_default_images:0',
  'index': 0,
  'shape': array([  1, 448, 448,   3], dtype=int32),
  'shape_signature': array([  1, 448, 448,   3], dtype=int32),
  'dtype': numpy.uint8,
  'quantization': (0.0078125, 127),
  'quantization_parameters': {'scales': array([0.0078125], dtype=float32),
   'zero_points': array([127], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [ ]:
'''Getting image dimension '''
rdim = (input_details[0]['shape'][1], input_details[0]['shape'][2])

In [ ]:
#Providing Path for Image
image_path = 'stop_cross_walk.jpg'

In [ ]:
input_data_type = input_details[0]["dtype"]
or_image = cv2.imread(image_path)
image = cv2.resize(or_image,rdim)
image = np.array(image, dtype=input_data_type)
image = np.expand_dims(image, axis = 0)

In [ ]:
or_image.shape

(607, 940, 3)

In [ ]:
output = signature_fn(images=image)
count = int(np.squeeze(output['output_0']))
scores = np.squeeze(output['output_1'])
classes = np.squeeze(output['output_2'])
boxes = np.squeeze(output['output_3'])

In [ ]:
results = []
for i in range(count):
  if scores[i] >= 0.3:        # Setting the confidence level above 0.3 to show bounding boxes
    result = {
      'bounding_box': boxes[i],
      'class_id': classes[i],
      'score': scores[i]
    }
    results.append(result)

In [ ]:
COLORS = np.random.randint(0, 254, size=(len(classes), 3), dtype=np.uint8)
for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * or_image.shape[1])
    xmax = int(xmax * or_image.shape[1])
    ymin = int(ymin * or_image.shape[0])
    ymax = int(ymax * or_image.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(or_image, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(labels[class_id], obj['score'] * 100)
    cv2.putText(or_image, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
original_uint8 = or_image.astype(np.uint8)

In [ ]:
cv2.imwrite('output.png', original_uint8)

True